In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")

In [5]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [7]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [9]:
df.shape

(57650, 4)

In [11]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [13]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [15]:
df.head(10)

,artist,song,text
0,Kim Wilde,Stay Awhile,I'm touching your face and I \r\nCan't get ve...
1,INXS,Viking Juice,If this was easy \r\nWhere would I be \r\nI ...
2,Lenny Kravitz,Black Velveteen,Black velveteen \r\nSimple and clean \r\nOh ...
3,Bruce Springsteen,Across The Border,"Uh, uh \r\nIt was the middle of June, me and ..."
4,Utopia,The Martyr,"Why, why was I born to know what I must know ..."
5,Kid Rock,Super Rhyme Maker,Bonnie shades \r\nAnd a high top fade \r\nSo...
6,Paul Simon,Sounds Of Silence,"Hello darkness, my old friend \r\nI've come t..."
7,Israel,Alive,Verse One: From one generation to another we d...
8,Waylon Jennings,Let Me Tell You My Mind,Some quiet morning let me let me tell you my m...
9,R. Kelly,Country Girl,"She was a country girl from Clebourne, Texas ..."


In [17]:
df['text'][0]

'I\'m touching your face and I  \r\nCan\'t get very far  \r\nYou travel a million miles  \r\nAlways in the stars  \r\nAnd crossing the clouds  \r\nA fading moon  \r\nWhispered "It\'s goodbye"  \r\nBut as it starts to die  \r\n  \r\nStay awhile  \r\nShow your love  \r\nChild among the stars  \r\nJust stay awhile  \r\nLay with me  \r\nTell me who you are  \r\n  \r\nI\'ve never believed in much  \r\nWhat can I be worth ?  \r\nI\'m always a shooting star  \r\nFalling down to earth  \r\nI prayed in my heart  \r\nYou\'d never know  \r\nFeelings wouldn\'t show  \r\nBut now before you go  \r\n  \r\nStay awhile  \r\nShow your love  \r\nChild among the stars  \r\nJust stay awhile  \r\nLay with me  \r\nTell me who you are  \r\n  \r\nAnd crossing the clouds  \r\nA fading moon  \r\nWhispers "It\'s goodbye"  \r\nBut as it starts to die  \r\n  \r\nStay awhile  \r\nShow your love  \r\nChild among the stars  \r\nJust stay awhile  \r\nLay with me  \r\nShow me who you are  \r\n  \r\nJust stay awhile\r\n\

In [19]:
 df = df.sample(5000)

In [21]:
df.shape

(5000, 3)

# Text Cleaning/ Text Preprocessing

In [24]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [26]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [28]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [34]:
similarity[0]

array([1.        , 0.00506487, 0.00964983, ..., 0.01055995, 0.00738443,
       0.00546232])

In [36]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [38]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [44]:
def recommendation(song_df):
    # Try to find the song index
    try:
        idx = df[df['song'].str.strip().str.lower() == song_df.strip().str.lower()].index[0]
    except IndexError:
        print(f"Song '{song_df}' not found in the DataFrame.")
        return []
    
    # Compute similarity and get recommendations
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    
    songs = []
    for i in distances[1:6]:  # Get the top 5 similar songs, excluding the first one (itself)
        songs.append(df.iloc[i[0]]['song'])
    
    return songs


# For Model Deployment Apply Pickle Method For Saving and Loading the model 

In [47]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))